### rackcli analytics - h4x VCV Rack off-radar analytics

There doesn't appear to be any analytics going on for VCV Rack plugin modules at the module level. The goal of this experiment is to aggreate a corpus of "available" patches and generate helpful analyitics (e.g. popularity, use cases, typical wiring/composition, etc.. Popular repositories of patches will be "considered" in leu of public APIs. ;-)

Here's an initial list of sources (both easy, and not so much):

1. PatchStorage
1. Facebook
1. Google
1. Youtube
1. Soudcloud

Usual disclaimers apply, this is a technical exercise, not suggested for commercial leverage -- you're on your own here, don't try this at home, etc..

In [2]:
from pprint import pprint as pp
from typing import List, Dict
from pathlib import Path
import os, json, requests

home = str(Path.home())
rackdir = home + '/Desktop/h4x/_queue_/vcv/Rack'
patchStorageURL = 'https://patchstorage.com/platform/vcv-rack/'

First, let's take a look at a snapshot of my currently installed patches.

In [38]:
plugins = json.load(open(rackdir + '/catalog.json'))['plugins']
print(f"{len(plugins)} plugins, {sum([p['modelCount'] for p in plugins])} modules installed")
print(sorted([p['slug'] for p in plugins], key=lambda s: s.lower()))
print(sorted([m['name'] for p in plugins for m in p['models']], key=lambda s: s.lower()))

111 plugins, 879 modules installed
['Aepelzens Modules', 'Alikins', 'alto777_LFSR', 'AmalgamatedHarmonics', 'AnimatedCircuits_Noises', 'AnimatedCircuits_Panimated', 'ArableInstruments', 'AS', 'AS-Drums-n-Filters', 'AS-Seqs-n-Tools', 'AudibleInstruments', 'AudibleInstrumentsPreview', 'Autinn', 'Autodafe - REDs', 'BaconMusic', 'Befaco', 'Bidoo', 'Blamsoft-XFXDistortionPack', 'Blamsoft-XFXF35', 'Blamsoft-XFXReverb', 'Blamsoft-XFXWave', 'Bogaudio', 'BOKONTEPByteBeatMachine', 'cf', 'CharredDesert', 'com-soundchasing-stochasm', 'Core', 'dBiz', 'DevilsHand', 'DHE-Modules', 'DLwigglz', 'DrRes', 'DrumKit', 'ErraticInstruments', 'ESeries', 'Floats', 'FrozenWasteland', 'FrozenWastelandKF', 'Fundamental', 'Gratrix', 'Grayscale', 'HetrickCV', 'Hora Processors', 'Hora-AnalogDrums', 'Hora-Mixers', 'Hora-Modulation', 'Hora-Processors', 'Hora-Sequencers', 'Hora-VCO_VCF_VCA', 'HotBunny', 'huaba', 'ImpromptuModular', 'JW-Modules', 'KlirrFactory', 'KlirrFactoryBS', 'KlirrFactoryC', 'KlirrFactoryFree', 'Kl

Let's start with [PatchStorage](https://patchstorage.com/platform/vcv-rack/). I reached out to them in early May, 2018 asking about an API, and they said it was on the drawing board, but had no published target date. While an account is required to upload patches, downloading is a publically accessible, anonymous function.

The trick is to page down through the results to trigger the progressive page load (CTRL+DOWN_ARROW), then traverse the anchor tags to grab the URLs, then download.

In [11]:
patches = requests.get(patchStorageURL) 
patches.content # no love here, try headless chrome + selenium (as expected)

b'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>403 Forbidden</title>\n</head><body>\n<h1>Forbidden</h1>\n<p>You don\'t have permission to access /platform/vcv-rack/\non this server.<br />\n</p>\n</body></html>\n'

In [24]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.binary_location = '/Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary'

driver = webdriver.Chrome(executable_path=home + '/Downloads/chromedriver', chrome_options=chrome_options)  
driver.get(patchStorageURL)

In [31]:
print(driver.title)

# patch links are in <a class="item-title" href="w00t!"/> instances, CMD+DOWNARROW, 'Loading...'
for link in driver.find_elements_by_css_selector('a.item-title'): print(link.get_attribute('href'))

VCV-Rack | Platforms | Patch Storage
https://patchstorage.com/talking-rackheads-episode-10/
https://patchstorage.com/wind-sweep-v-2/
https://patchstorage.com/smoke-in-your-ears-with-drums/
https://patchstorage.com/cloud-generator-clap/
https://patchstorage.com/2011-bells-with-drums/
https://patchstorage.com/smoke-gets-in-your-ears-quantussy-cell-13/
https://patchstorage.com/tonenot/
https://patchstorage.com/quantussy-hair-pick/
https://patchstorage.com/fantasies/
https://patchstorage.com/xcobassfolder/
https://patchstorage.com/more-saturday-adventures/
https://patchstorage.com/luci-cell-07/
https://patchstorage.com/wind-sweep/
https://patchstorage.com/braids-synthesis-modules-series-11-swarm-of-7-sawtooth-waves/
https://patchstorage.com/paint-dripping/
https://patchstorage.com/patching-without-words-8/
https://patchstorage.com/improvised-patch-tutorial-18-valley-topograph-aka-grids/
https://patchstorage.com/mega-patch-pack-2/


In [32]:
driver.close()

In [34]:
patchPageURL = 'https://patchstorage.com/talking-rackheads-episode-10/'
driver = webdriver.Chrome(executable_path=home + '/Downloads/chromedriver', chrome_options=chrome_options)  
driver.get(patchPageURL)

In [40]:
print(driver.title)
patchURL = driver.find_element_by_id('DownloadPatch').get_attribute('href')

Talking Rackheads Episode #10 | Patch Storage


'https://patchstorage.com/wp-content/uploads/2018/05/Episode-10-patch.vcv'

In [44]:
driver.close()

In [45]:
patchURL = 'https://patchstorage.com/wp-content/uploads/2018/05/Episode-10-patch.vcv'
chromeUserAgent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'
patch = requests.get(patchURL, headers={'User-Agent': chromeUserAgent}) # lookup via chrome debugger: navigator.userAgent

In [50]:
print(patch.content.decode('utf-8'))

{
  "version": "0.6.0",
  "modules": [
    {
      "plugin": "Core",
      "version": "0.6.0",
      "model": "AudioInterface",
      "pos": [
        104,
        0
      ],
      "params": [],
      "data": {
        "audio": {
          "driver": 5,
          "deviceName": "Apple Inc.: Built-in Output",
          "offset": 0,
          "maxChannels": 8,
          "sampleRate": 44100,
          "blockSize": 256
        }
      }
    },
    {
      "plugin": "VCV-Console",
      "version": "0.6.0",
      "model": "Console8_3",
      "pos": [
        18,
        0
      ],
      "params": [
        {
          "paramId": 0,
          "value": 1.0
        },
        {
          "paramId": 8,
          "value": 0.0
        },
        {
          "paramId": 16,
          "value": 0.0
        },
        {
          "paramId": 24,
          "value": -1.0
        },
        {
          "paramId": 32,
          "value": 0.364500046
        },
        {
          "paramId": 40,
          "valu

Lots of patches are shared on Facebook in both the official and official groups.

Lots of action crawled by Google as well.

And don't forget finished goods on Youtube, though it's non-trivial to dig into the particulars, but of course it's worth a try! ;-)

And finally (for now), let's hit up Soundcloud.